In [1]:
import time
import re
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [6]:
df = pd.read_csv('sirket_websiteleri_sonuclar0_25000.csv')  # 'header=None' sütun adlarının olmadığını belirtir

# İlk 100 satır
df = df.iloc[24000:25000]

# "Site bulunamadı" olan hücreleri NaN ile değiştirme
df['website'] = df['website'].replace('Site bulunamadı', np.nan)

# Sonuçları saklamak için bir liste
company_info = []
site_count = 0  # Toplam site sayısını takip etmek için sayaç

# Tarayıcıyı başlat
driver = webdriver.Edge()  # options olmadan tarayıcıyı doğrudan başlatır
driver.get("https://www.google.com")
time.sleep(2)

# Her bir site için Google dork sorgusu yap ve snippet'leri çek
for idx, row in df.iterrows():
    try:
        # Her 10 aramada bir tarayıcıyı kapatıp yeniden başlat
        if site_count > 0 and site_count % 10 == 0:
            driver.quit()  # Tarayıcıyı kapat
            print(f"{site_count} site tarandı, 10 saniye bekleniyor...")
            time.sleep(10)  # 10 saniye bekle
            # Tarayıcıyı yeniden başlat
            driver = webdriver.Edge()  # options olmadan yeniden başlatır
            driver.get("https://www.google.com")
            time.sleep(2)

        # Arama çubuğunu bulun ve önceki sorguyu temizleyin
        search_box = driver.find_element(By.NAME, "q")
        search_box.clear()
        
        # Dork sorgusunu girin ve arama yapın
        dork_query = f'{row["website"]} "contact" OR "phone" OR "email" OR "kontakt" OR "telephone"'
        search_box.send_keys(dork_query)
        search_box.send_keys(Keys.RETURN)
        time.sleep(3)  # Arama sonuçlarının yüklenmesi için bekleyin

        # Snippet'leri bul ve metinlerini topla
        results = driver.find_elements(By.CSS_SELECTOR, 'div.g')  # Tüm arama sonuçlarını al
        snippet_texts = []

        # Her bir arama sonucunun metnini alın
        for result in results[:3]:  # İlk 3 sonucu kontrol edin
            snippet_texts.append(result.text)  # Sonucun tamamını metin olarak kaydedin

        # Sonuçları yeni DataFrame'e eklemek için kaydedin
        company_info.append({
            "sirket_adi": row["sirket_adi"],
            "website": row["website"],
            "snippets": "\n\n".join(snippet_texts)  # Snippet'leri birleştirip tek bir hücreye ekleyin
        })

        # Tarama yapılan site sayısını artır
        site_count += 1

    except Exception as e:
        print(f"Hata oluştu: {row['sirket_adi']} için {e}")
        company_info.append({
            "sirket_adi": row["sirket_adi"],
            "website": row["website"],
            "snippets": None
        })

# Döngü tamamlandıktan sonra tarayıcıyı kapat
driver.quit()

# Sonuçları bir DataFrame'e dönüştür ve kaydet
result_df = pd.DataFrame(company_info)

# E-posta ve telefon numaraları için iyileştirilmiş regex desenleri
email_regex = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
phone_regex = r"\+?\d{1,3}?[-.\s]?\(?\d{2,4}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,4}[-.\s]?\d{1,9}"

# E-posta ve telefon bilgilerini çekip yeni sütunlar olarak ekleme
result_df['email'] = result_df['snippets'].apply(lambda x: re.findall(email_regex, x)[0] if re.findall(email_regex, x) else None)
result_df['telefon'] = result_df['snippets'].apply(lambda x: re.findall(phone_regex, x)[0] if re.findall(phone_regex, x) else None)

# Dosya adı değişkenini tanımlayın (daha sonra kolayca değiştirilebilir)
csv_file_name = 'sirket_websiteleri_dork_sonuclar24000_25000.csv'

result_df.to_csv(csv_file_name, index=False, encoding='utf-8')


10 site tarandı, 10 saniye bekleniyor...
20 site tarandı, 10 saniye bekleniyor...
30 site tarandı, 10 saniye bekleniyor...
40 site tarandı, 10 saniye bekleniyor...
50 site tarandı, 10 saniye bekleniyor...
60 site tarandı, 10 saniye bekleniyor...
70 site tarandı, 10 saniye bekleniyor...
80 site tarandı, 10 saniye bekleniyor...
90 site tarandı, 10 saniye bekleniyor...
100 site tarandı, 10 saniye bekleniyor...
110 site tarandı, 10 saniye bekleniyor...
120 site tarandı, 10 saniye bekleniyor...
130 site tarandı, 10 saniye bekleniyor...
140 site tarandı, 10 saniye bekleniyor...
150 site tarandı, 10 saniye bekleniyor...
160 site tarandı, 10 saniye bekleniyor...
170 site tarandı, 10 saniye bekleniyor...
180 site tarandı, 10 saniye bekleniyor...
190 site tarandı, 10 saniye bekleniyor...
200 site tarandı, 10 saniye bekleniyor...
210 site tarandı, 10 saniye bekleniyor...
220 site tarandı, 10 saniye bekleniyor...
230 site tarandı, 10 saniye bekleniyor...
240 site tarandı, 10 saniye bekleniyor...
2